## PARCEIROS 360

###  Pagamento - Utilização Factoring & Bloqueios

- <b>Analista(s)</b>: Daniel Rezende
- <b>Projeto</b>: Parceiros 360 - Um modelo de análise de risco para auxiliar no controle das tomadas de decisões através de análises preditivas
- <b>Data</b>: 10/12/2022
- <b>Descrição</b>: 

In [ ]:
!python --version

### BIBLIOTECAS:

In [ ]:
# !pip install treeinterpreter

In [ ]:
## data
import pandas as pd
import numpy as np
import datetime as dt
from dateutil.relativedelta import relativedelta
from unicodedata import normalize

## vis
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

## modelo
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier

from treeinterpreter import treeinterpreter as ti

## alertas
import warnings
warnings.filterwarnings("ignore")

### LAYOUT:

In [ ]:
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (12, 6)

pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 1000)

rc={'font.size': 16, 'axes.labelsize': 16, 'legend.fontsize': 16,
    'axes.titlesize': 18, 'xtick.labelsize': 16, 'ytick.labelsize': 16}

sns.set(rc=rc)
sns.set_style("darkgrid")
sns.set_palette("pastel")

### CONFIGS:

In [ ]:
#pd.set_option('max_columns', 200)
pd.options.display.max_columns = 200 

seed = 42
np.random.seed(seed)

### FUNÇÕES

In [ ]:
#SE PRECISAR DO ONE HOT USAR ANTES DO SCALERS PRA NAO DAR ERRO

def scalers(base):
    nova_base = base.copy()
    scaler = MinMaxScaler(feature_range=(0,1))
    for column in base.columns:
        aux = scaler.fit_transform(base[[column]])
        nova_base[column] = aux
    return nova_base

In [ ]:
#base_treino_scaled = BASE DE TREINO, SEM A COLUNA DO KPI
#DADOS = BASE COMPLETA, COM O TARGET
#KPI NOME = O NOME DA COLUNA ONDE ESTÁ O TARGET

def makedf(base_treino_scaled, dados, base_treino):
    
    kmeans = KMeans(n_clusters = 3, max_iter = 300, init = 'k-means++', n_init = 10, random_state = 0)
    clusters = kmeans.fit_predict(base_treino_scaled)

    model = RandomForestClassifier(random_state=0)
    model.fit(base_treino_scaled,clusters)
    prediction, bias, contributions = ti.predict(model, base_treino_scaled)

    contributions = np.round(contributions,5)*1000

    contributions_c1 = []
    contributions_c2 = []
    contributions_c3 = []
    contributions_filtered = []

    for i in range(len(contributions)):
        contributions_c1.append([contributions[i][0][0], contributions[i][1][0], contributions[i][2][0], contributions[i][3][0], contributions[i][4][0], contributions[i][5][0]])
        contributions_c2.append([contributions[i][0][1], contributions[i][1][1], contributions[i][2][1], contributions[i][3][1], contributions[i][4][1], contributions[i][5][1]])
        contributions_c3.append([contributions[i][0][2], contributions[i][1][2], contributions[i][2][2], contributions[i][3][2], contributions[i][4][2], contributions[i][5][2]])
    
    for i in range(len(clusters)):
        if clusters[i] == 0:
            contributions_filtered.append(contributions_c1[i])
        if clusters[i] == 1:
            contributions_filtered.append(contributions_c2[i])
        if clusters[i] == 2:
            contributions_filtered.append(contributions_c3[i])

    scaler = MinMaxScaler(feature_range=(0,100))
    contributions_filtered_scaled = scaler.fit_transform(contributions_filtered)

    contributions_filtered = pd.DataFrame(contributions_filtered_scaled, columns=base_treino_scaled.columns + '_CONTRIBUTION')

#     base_treino['KPI_original'] = dados[kpi_nome].values
    base_treino['resultado_modelo'] = clusters
    base_treino['resultado_modelo'] = base_treino['resultado_modelo'].map({0:'BOM', 1:'MÉDIO', 2:'RUIM'})

    base_treino.reset_index(inplace=True, drop=False)

    results = pd.concat([base_treino, contributions_filtered], axis=1)
    return results

### DADOS:

In [ ]:
PATH = "datasets/"

****
### BASE DE PAGAMENTO:

In [ ]:
# FILE = "/kaggle/input/datasetsnew/previsao_pagamentos.xlsx"
FILE = "previsao_pagamentos.xlsx"
pagto = pd.read_excel(FILE)
pagto['RAIZ_CNPJ'] = pagto['RAIZ_CNPJ'].apply(str)
pagto = pagto.rename(columns={'resultado_modelo': 'resultado_modelo_pagto'})
pagto = pagto[['RAIZ_CNPJ', 'COMPETENCIA', 'NOME_FORNECEDOR', 'resultado_modelo_pagto']].drop_duplicates().reset_index(drop=True)

pagto

***
### BASE DE FATURAMENTO:

In [ ]:
# FILE = "/kaggle/input/datasetsnew/previsao_faturamento.xlsx"
FILE = "previsao_faturamento.xlsx"
faturamento = pd.read_excel(FILE)
faturamento['RAIZ_CNPJ'] = faturamento['RAIZ_CNPJ'].apply(str)
faturamento = faturamento.rename(columns={'resultado_modelo': 'resultado_modelo_faturamento'})
faturamento = faturamento[['RAIZ_CNPJ', 'COMPETENCIA', 'NOME_FORNECEDOR', 'resultado_modelo_faturamento']]\
                           .drop_duplicates().reset_index(drop=True)

faturamento

***
### BASE COMISSÃO

In [ ]:
# FILE = "/kaggle/input/datasetsnew/previsao_comissao.xlsx"
# comissao = pd.read_excel(FILE)

FILE = "previsao_comissao.xlsx"
comissao = pd.read_excel(FILE)
comissao['RAIZ_CNPJ'] = comissao['RAIZ_CNPJ'].apply(str)
comissao = comissao.rename(columns={'resultado_modelo': 'resultado_modelo_comissao'})
comissao = comissao[['RAIZ_CNPJ', 'COMPETENCIA', 'NOME_FORNECEDOR', 'resultado_modelo_comissao']]\
                     .drop_duplicates().reset_index(drop=True)

comissao

***
### BASE ALIADOS

In [ ]:
# FILE = "/kaggle/input/datasetsnew/previsao_aliados_variacao_kpi1.xlsx"
# previsao_aliados_variacao_kpi1 = pd.read_excel(FILE)

FILE = "previsao_aliados_variacao_kpi1.xlsx"
previsao_aliados_variacao_kpi1 = pd.read_excel(FILE)
previsao_aliados_variacao_kpi1['COMPETENCIA'] = pd.to_datetime(previsao_aliados_variacao_kpi1['COMPETENCIA'])
previsao_aliados_variacao_kpi1 = previsao_aliados_variacao_kpi1.rename(columns={'resultado_modelo': 'resultado_modelo_previsao_aliados_variacao_kpi1'})
previsao_aliados_variacao_kpi1 = previsao_aliados_variacao_kpi1[['RAIZ_CNPJ', 'COMPETENCIA', 'NOME_FORNECEDOR', 'resultado_modelo_previsao_aliados_variacao_kpi1']]\
                                                               .drop_duplicates().reset_index(drop=True)

previsao_aliados_variacao_kpi1

In [ ]:
# FILE = "/kaggle/input/datasetsnew/previsao_aliados_trabalhista_kpi2.xlsx"
# previsao_aliados_trabalhista_kpi2 = pd.read_excel(FILE)

FILE = "previsao_aliados_trabalhista_kpi2.xlsx"
previsao_aliados_trabalhista_kpi2 = pd.read_excel(FILE)
previsao_aliados_trabalhista_kpi2['COMPETENCIA'] = pd.to_datetime(previsao_aliados_trabalhista_kpi2['COMPETENCIA'])
previsao_aliados_trabalhista_kpi2 = previsao_aliados_trabalhista_kpi2.rename(columns={'resultado_modelo': 'resultado_modelo_previsao_aliados_trabalhista_kpi2'})
previsao_aliados_trabalhista_kpi2 = previsao_aliados_trabalhista_kpi2[['RAIZ_CNPJ', 'COMPETENCIA', 'NOME_FORNECEDOR', 'resultado_modelo_previsao_aliados_trabalhista_kpi2']]\
                                                                     .drop_duplicates().reset_index(drop=True)

previsao_aliados_trabalhista_kpi2

In [ ]:
# FILE = "/kaggle/input/datasetsnew/previsao_aliados_endividamento_kpi3.xlsx"
# previsao_aliados_endividamento_kpi3 = pd.read_excel(FILE)

FILE = "previsao_aliados_endividamento_kpi3.xlsx"
previsao_aliados_endividamento_kpi3 = pd.read_excel(FILE)
previsao_aliados_endividamento_kpi3['COMPETENCIA'] = pd.to_datetime(previsao_aliados_endividamento_kpi3['COMPETENCIA'])
previsao_aliados_endividamento_kpi3 = previsao_aliados_endividamento_kpi3.rename(columns={'resultado_modelo': 'resultado_modelo_previsao_aliados_endividamento_kpi3'})
previsao_aliados_endividamento_kpi3 = previsao_aliados_endividamento_kpi3[['RAIZ_CNPJ', 'COMPETENCIA', 'NOME_FORNECEDOR', 'resultado_modelo_previsao_aliados_endividamento_kpi3']]\
                                                                         .drop_duplicates().reset_index(drop=True)

previsao_aliados_endividamento_kpi3

***
### JOINS:

#### JOIN pagamento vs faturamento:

In [ ]:
for_pgm = pagto.merge(faturamento, 
                      on=['RAIZ_CNPJ', 'COMPETENCIA', 'NOME_FORNECEDOR'], 
                      how='outer')\
               .drop_duplicates().reset_index(drop=True)

for_pgm

#### JOIN pagamento vs faturamento vs comissao:

In [ ]:
for_pgm_com = for_pgm.merge(comissao,                       
                            on=['RAIZ_CNPJ', 'COMPETENCIA', 'NOME_FORNECEDOR'], 
                            how='outer')\
                     .drop_duplicates().reset_index(drop=True)
for_pgm_com

#### JOIN pagamento vs faturamento vs comissao vs aliados

In [ ]:
for_pgm_com_ali1 = for_pgm_com.merge(previsao_aliados_variacao_kpi1, 
                                     on=['RAIZ_CNPJ', 'COMPETENCIA', 'NOME_FORNECEDOR'], 
                                     how='outer')\
                              .drop_duplicates().reset_index(drop=True)


for_pgm_com_ali1

In [ ]:
for_pgm_com_ali2 = for_pgm_com_ali1.merge(previsao_aliados_trabalhista_kpi2, 
                                          on=['RAIZ_CNPJ', 'COMPETENCIA', 'NOME_FORNECEDOR'], 
                                          how='outer')\
                                   .drop_duplicates().reset_index(drop=True)

for_pgm_com_ali2

In [ ]:
for_pgm_com_ali3 = for_pgm_com_ali2.merge(previsao_aliados_endividamento_kpi3, 
                                          on=['RAIZ_CNPJ', 'COMPETENCIA', 'NOME_FORNECEDOR'], 
                                          how='outer')\
                                   .drop_duplicates().reset_index(drop=True)

for_pgm_com_ali3.head(7)

In [ ]:
# fill string 0 para NaN
for_pgm_com_ali3['resultado_modelo_pagto'] = for_pgm_com_ali3['resultado_modelo_pagto'].fillna(value='0')
for_pgm_com_ali3['resultado_modelo_faturamento'] = for_pgm_com_ali3['resultado_modelo_faturamento'].fillna(value='0')
for_pgm_com_ali3['resultado_modelo_comissao'] = for_pgm_com_ali3['resultado_modelo_comissao'].fillna(value='0')
for_pgm_com_ali3['resultado_modelo_previsao_aliados_variacao_kpi1'] = for_pgm_com_ali3['resultado_modelo_previsao_aliados_variacao_kpi1'].fillna(value='0')
for_pgm_com_ali3['resultado_modelo_previsao_aliados_trabalhista_kpi2'] = for_pgm_com_ali3['resultado_modelo_previsao_aliados_trabalhista_kpi2'].fillna(value='0')
for_pgm_com_ali3['resultado_modelo_previsao_aliados_endividamento_kpi3'] = for_pgm_com_ali3['resultado_modelo_previsao_aliados_endividamento_kpi3'].fillna(value='0')

for_pgm_com_ali3

for_pgm_com_ali3.head(7)

***
### ORDINAL ENCODING

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

for_pgm_com_ali3_oe = OrdinalEncoder().fit_transform(for_pgm_com_ali3[['resultado_modelo_pagto', 
                                                                       'resultado_modelo_faturamento', 
                                                                       'resultado_modelo_comissao', 
                                                                       'resultado_modelo_previsao_aliados_variacao_kpi1', 
                                                                       'resultado_modelo_previsao_aliados_trabalhista_kpi2', 
                                                                       'resultado_modelo_previsao_aliados_endividamento_kpi3']])

for_pgm_com_ali3['resultado_modelo_pagto_oe'] = for_pgm_com_ali3_oe[:, 0]

for_pgm_com_ali3['resultado_modelo_faturamento_oe'] = for_pgm_com_ali3_oe[:, 1]

for_pgm_com_ali3['resultado_modelo_comissao_oe'] = for_pgm_com_ali3_oe[:, 2]

for_pgm_com_ali3['resultado_modelo_previsao_aliados_variacao_kpi1_oe'] = for_pgm_com_ali3_oe[:, 3]

for_pgm_com_ali3['resultado_modelo_previsao_aliados_trabalhista_kpi2_oe'] = for_pgm_com_ali3_oe[:, 4]

for_pgm_com_ali3['resultado_modelo_previsao_aliados_endividamento_kpi3_oe'] = for_pgm_com_ali3_oe[:, 5]

In [ ]:
for_pgm_com_ali3['resultado_modelo_faturamento_oe'].unique()


In [ ]:
for_pgm_com_ali3

In [ ]:
dados = for_pgm_com_ali3.copy()
dados.set_index(['RAIZ_CNPJ', 'COMPETENCIA', 'NOME_FORNECEDOR'], inplace=True)
dados

In [ ]:
dados.columns

In [ ]:
base_treino_KPI_A1_variacao = dados[['resultado_modelo_pagto_oe', 
                                     'resultado_modelo_faturamento_oe', 
                                     'resultado_modelo_comissao_oe', 
                                     'resultado_modelo_previsao_aliados_variacao_kpi1_oe', 
                                     'resultado_modelo_previsao_aliados_trabalhista_kpi2_oe', 
                                     'resultado_modelo_previsao_aliados_endividamento_kpi3_oe']]

In [ ]:
base_treino_KPI_A1_variacao_scaled = scalers(base_treino_KPI_A1_variacao)

base_treino_KPI_A1_variacao_scaled

***
### CLUSTERING COM K-Means:

In [ ]:
#base_treino_scaled = BASE DE TREINO, SEM A COLUNA DO KPI, scaled
#DADOS = BASE COMPLETA, COM O TARGET
#KPI NOME = O NOME DA COLUNA ONDE ESTÁ O TARGET
#base_treino_KPI_A1_variacao = BASE DE TREINO, SEM A COLUNA DO KPI


results_kpi = makedf(base_treino_KPI_A1_variacao_scaled, dados, base_treino_KPI_A1_variacao)
results_kpi

In [ ]:
results_kpi = results_kpi.drop_duplicates().reset_index(drop=True)
results_kpi

In [ ]:
results_kpi.to_excel(r'previsao_all.xlsx')